In [1]:
import pandas as pd

In this project, I'll be looking at the precipitation levels across the contiguous United States from 1917 to 2017. I am interested in seeing how the landscape has changed in this aspect, especially in light of human-caused climate change, but also just naturally over time. 

I sourced data (with great pain) from this website: 

https://www7.ncdc.noaa.gov/CDO/CDODivisionalSelect.jsp

I sourced the divisional data (which breaks the each state up into climate divisions), but I wasn't able to use it, because I couldn't find shapefile data in JSON format. Instead, I averaged the divisional data across each state, which is maybe a little questionable but it seems to roughly check out when cross-referenced with the state data.

Reading the data (Alaska and Hawaii are removed):

In [2]:
states = ["Alabama", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", 
"Delaware", "Florida",  "Georgia", "Idaho", "Illinois", "Indiana", "Iowa",
"Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts",
"Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", 
"Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina",
"North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island",
"South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont",
"Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]

dfs = []
for state in states:
    url = "climate_data/" + state + ".txt"
    dfs.append(pd.read_csv(url))
    
state_df = pd.concat(dfs)

Double-checking that I have all 48 contiguous states:

In [3]:
state_df['StateCode'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48])

Exploring the data and extracting the parts I want:

In [6]:
print(state_df.columns)
dates = list(state_df['YearMonth'].unique())
print(dates)
date_df = state_df[['StateCode', 'Division', 'YearMonth', '    PCP']]
date_df

Index(['StateCode', 'Division', 'YearMonth', '    PCP', '   TAVG', '   PDSI',
       '   PHDI', '   ZNDX', '   PMDI', '    CDD', '    HDD', '   SP01',
       '   SP02', '   SP03', '   SP06', '   SP09', '   SP12', '   SP24',
       '   TMIN', '   TMAX', 'Unnamed: 20'],
      dtype='object')
[191701, 191702, 191703, 191704, 191705, 191706, 191707, 191708, 191709, 191710, 191711, 191712, 191801, 191802, 191803, 191804, 191805, 191806, 191807, 191808, 191809, 191810, 191811, 191812, 191901, 191902, 191903, 191904, 191905, 191906, 191907, 191908, 191909, 191910, 191911, 191912, 192001, 192002, 192003, 192004, 192005, 192006, 192007, 192008, 192009, 192010, 192011, 192012, 192101, 192102, 192103, 192104, 192105, 192106, 192107, 192108, 192109, 192110, 192111, 192112, 192201, 192202, 192203, 192204, 192205, 192206, 192207, 192208, 192209, 192210, 192211, 192212, 192301, 192302, 192303, 192304, 192305, 192306, 192307, 192308, 192309, 192310, 192311, 192312, 192401, 192402, 192403, 192404, 1924

,StateCode,Division,YearMonth,PCP
0,1,7,191701,5.69
1,1,8,191701,3.89
2,1,4,191701,8.15
3,1,3,191701,7.46
4,1,2,191701,7.15
5,1,6,191701,7.33
6,1,1,191701,6.21
7,1,5,191701,7.24
8,1,8,191702,3.95
9,1,1,191702,4.53


I want a datapoint for each state for each date. I will do that by averaging all the divisional `PCP` values for each date and appending them to an two dimensional array with the format `[date, state (by StateCode), PCP value]`

In [7]:
avgs = []
for i in range(len(dates)):
    date_df = state_df.loc[state_df['YearMonth'] == dates[i]]
    date_df = date_df[['StateCode', 'Division', 'YearMonth', '    PCP']]
    for j in range (1, 49):
        my_sum = 0
        count = 0
        my_state_df = date_df.loc[date_df['StateCode'] == j]
        for index, row in my_state_df.iterrows():
            my_sum += row['    PCP']
            count += 1
        avg = "none"
        if count > 0:
            avg = my_sum/count
        avgs.append([dates[i], j, avg])
avgs

[[191701, 1, 6.6400000000000006],
 [191701, 2, 2.5842857142857141],
 [191701, 3, 3.7488888888888883],
 [191701, 4, 2.6599999999999997],
 [191701, 5, 0.92200000000000004],
 [191701, 6, 3.1999999999999997],
 [191701, 7, 3.0600000000000001],
 [191701, 8, 1.3971428571428572],
 [191701, 9, 5.2500000000000009],
 [191701, 10, 2.0739999999999998],
 [191701, 11, 1.9766666666666666],
 [191701, 12, 3.1400000000000001],
 [191701, 13, 0.83333333333333348],
 [191701, 14, 0.30222222222222217],
 [191701, 15, 6.2825000000000006],
 [191701, 16, 5.6022222222222213],
 [191701, 17, 4.3033333333333337],
 [191701, 18, 3.1875],
 [191701, 19, 3.1366666666666667],
 [191701, 20, 1.1330000000000002],
 [191701, 21, 1.1766666666666665],
 [191701, 22, 6.1749999999999998],
 [191701, 23, 1.9033333333333333],
 [191701, 24, 1.3185714285714287],
 [191701, 25, 0.54749999999999999],
 [191701, 26, 0.89250000000000007],
 [191701, 27, 3.27],
 [191701, 28, 3.2066666666666666],
 [191701, 29, 0.79249999999999998],
 [191701, 30, 

Finally, I write the data out as a CSV to use in my data visualization.

In [ ]:
with open('avgs.csv', 'w') as f:
    f.write('date,state,avgpcp\n')
    for item in avgs:
        f.write(str(item[0]) + "," + str(item[1]) + "," + str(item[2]) + "\n")
    f.close()